# Plan of attack:
Use a subset of the million song [dataset](http://millionsongdataset.com/pages/getting-dataset/) to create the training data for NER
DFinetune the english spacy NER model to recognize custom labels of [ARTIST, SONG, FEATURE]

Resources:
https://github.com/RohanBhirangi/Million-Song-Dataset-Analysis

https://towardsdatascience.com/custom-named-entity-recognition-using-spacy-7140ebbb3718

https://spacy.io/usage/training#training-data

https://methodmatters.github.io/editing-id3-tags-mp3-meta-data-in-python/


In [3]:
import sys
sys.path.append('/Users/petr/Documents/fun_stuff/auto_song_labeller/modules')
import hdf5_getters as getters
import numpy as np
import pandas as pd
import os
import fnmatch
import pickle

In [2]:
# # script to cache the dataset into a csv file
# import sys
# sys.path.append('/Users/petr/Documents/fun_stuff/youtube_app/modules')
# import hdf5_getters as getters
# import numpy as np
# import h5py
# import pandas as pd
# import os
# import fnmatch

# df = pd.DataFrame(columns=['Song Name', 'Artist'])

# matches = list()
# df_final = pd.DataFrame()
# f= open("../data/ner/MSD.csv","w")
# for root,dirnames,filenames in os.walk('/Users/petr/Documents/fun_stuff/youtube_app/data/ner/MillionSongSubset'):
#     for filename in fnmatch.filter(filenames, '*.h5'):
#         #print(os.path.join(root, filename))
#         hdf = pd.HDFStore(os.path.join(root, filename),mode ='r')
#         artist = getters.get_artist_name(hdf).decode('UTF-8')
#         #print(artist)
#         name = getters.get_title(hdf).decode('UTF-8')
#         #print(name)
#         df1 = pd.DataFrame({'Song Name': [name], 'Artist': [artist]})
#         df = pd.concat([df, df1], ignore_index=True, axis=0)
#         hdf.close()
# f.close()

# df.to_csv("../data/ner/MSD.csv")
# df.head()

In [4]:
df = pd.read_csv("../data/MSD.csv", index_col=0)
df

,Song Name,Artist
0,Je Sais Que La Terre Est Plate,Raphaël
1,On Efface,Julie Zenatti
2,Howells Delight,The Baltimore Consort
3,Martha Served,I Hate Sally
4,Zip-A-Dee-Doo-Dah (Song of the South),Orlando Pops Orchestra
...,...,...
9995,One About Heaven,Brent Lamb
9996,October,U2
9997,Comin' Home,ZO2
9998,Pode Me Chamar,Eddie


In [28]:
def clean_text(text):
    # Remove non-alphanumeric characters
    text = re.sub('[^a-zA-Z\- ]', '', text)

    # lowercase text 
    text = text.lower()
    
    return text

In [145]:
# preprocessing data into correct form for spacy
# following tutorial
import spacy
from spacy.tokens import DocBin
import random
import json
import re

trainData = []
raw_text = []

entities = {"SONG": "ARTIST"}

for idx, row in df.iterrows():
    song = str(row[0]).strip()# turn song name into a string in case it isn't, and strip leading/trailing whitespace
    #song = clean_text(song) # remove non alphanumeric characters and make lowercase
    artist = str(row[1]).strip() # same for artist name
    #artist = clean_text(artist) # remove non alphanumeric characters and make lowercase
    
    try: # fetching the length of the song name if it exists
        len_song = len(song)
    except:
        len_song = 0
    try: # fetching the length of the artist name if it exists    
        len_artist = len(artist)
    except:
        len_artist = 0
    
    #text = song + ' - ' + artist
    songFirst = True
    if len_song != 0 and len_artist != 0:
        if random.random() <= 0.5: # making it so the artist is first 50% of the time
            cleanText = song + ' - ' + artist
        else:
            cleanText = artist + ' - ' + song
            songFirst = False
    else:
        cleanText = song
        
    if random.random() <= 0.05: # adding some non-entity strings to roughly match the frequency in youtube videos
        cleanText += ' (OFFICIAL MUSIC VIDEO)'
    dict_value = []
    
    raw_text.append(cleanText)
    # In the case where neither song nor artist is None:
    if song != None and artist != None:
        ents = []
        for songEnt, artistEnt in entities.items():
            if songFirst:
                ent_song = (0, len_song, songEnt)
                ents.append(ent_song)

                ent_artist = (len_song+3, len_song+3+len_artist, artistEnt) # with dash
                ents.append(ent_artist)
            else:
                ent_song = (len_artist+3, len_artist+3+len_song, songEnt)
                ents.append(ent_song)

                ent_artist = (0, len_artist, artistEnt) # with dash
                ents.append(ent_artist)

        
    # for the case where there is only a song name:
    elif song != None:
        ents = []
        for songEnt, _ in entities.items():
            ent_song = (0, len_song, songEnt)
            print('4')
            print(ent_song)
            ents.append(ent_song)
    # for the case there is only an artist somehow:
    else:
        ents = []
        for _, artistEnt in entities.items():
            ent_artist = (0, len_artist, artistEnt) # with dash

            ents.append(ent_artist)

    print(ents)
    entsDict = {}
    entsDict["entities"] = ents
    trainingExample = (cleanText, entsDict)
    trainData.append(trainingExample)

with open('/Users/petr/Documents/fun_stuff/youtube_app/data/ner/nerData_list.txt', 'w') as f:
    f.write(json.dumps(trainData))

[(10, 40, 'SONG'), (0, 7, 'ARTIST')]
[(0, 9, 'SONG'), (12, 25, 'ARTIST')]
[(0, 15, 'SONG'), (18, 39, 'ARTIST')]
[(0, 13, 'SONG'), (16, 28, 'ARTIST')]
[(0, 37, 'SONG'), (40, 62, 'ARTIST')]
[(10, 52, 'SONG'), (0, 7, 'ARTIST')]
[(0, 40, 'SONG'), (43, 51, 'ARTIST')]
[(8, 57, 'SONG'), (0, 5, 'ARTIST')]
[(16, 25, 'SONG'), (0, 13, 'ARTIST')]
[(22, 47, 'SONG'), (0, 19, 'ARTIST')]
[(0, 34, 'SONG'), (37, 43, 'ARTIST')]
[(11, 17, 'SONG'), (0, 8, 'ARTIST')]
[(0, 20, 'SONG'), (23, 35, 'ARTIST')]
[(0, 10, 'SONG'), (13, 25, 'ARTIST')]
[(26, 45, 'SONG'), (0, 23, 'ARTIST')]
[(0, 15, 'SONG'), (18, 29, 'ARTIST')]
[(18, 48, 'SONG'), (0, 15, 'ARTIST')]
[(0, 24, 'SONG'), (27, 38, 'ARTIST')]
[(16, 22, 'SONG'), (0, 13, 'ARTIST')]
[(17, 35, 'SONG'), (0, 14, 'ARTIST')]
[(28, 85, 'SONG'), (0, 25, 'ARTIST')]
[(15, 29, 'SONG'), (0, 12, 'ARTIST')]
[(17, 28, 'SONG'), (0, 14, 'ARTIST')]
[(32, 48, 'SONG'), (0, 29, 'ARTIST')]
[(8, 16, 'SONG'), (0, 5, 'ARTIST')]
[(0, 14, 'SONG'), (17, 21, 'ARTIST')]
[(0, 22, 'SONG'), (2

[(0, 11, 'SONG'), (14, 29, 'ARTIST')]
[(6, 25, 'SONG'), (0, 3, 'ARTIST')]
[(0, 14, 'SONG'), (17, 27, 'ARTIST')]
[(0, 30, 'SONG'), (33, 43, 'ARTIST')]
[(0, 13, 'SONG'), (16, 30, 'ARTIST')]
[(0, 15, 'SONG'), (18, 25, 'ARTIST')]
[(24, 52, 'SONG'), (0, 21, 'ARTIST')]
[(0, 30, 'SONG'), (33, 44, 'ARTIST')]
[(0, 14, 'SONG'), (17, 24, 'ARTIST')]
[(0, 27, 'SONG'), (30, 43, 'ARTIST')]
[(15, 23, 'SONG'), (0, 12, 'ARTIST')]
[(0, 13, 'SONG'), (16, 34, 'ARTIST')]
[(15, 24, 'SONG'), (0, 12, 'ARTIST')]
[(0, 6, 'SONG'), (9, 26, 'ARTIST')]
[(0, 10, 'SONG'), (13, 26, 'ARTIST')]
[(0, 7, 'SONG'), (10, 19, 'ARTIST')]
[(0, 40, 'SONG'), (43, 55, 'ARTIST')]
[(0, 10, 'SONG'), (13, 31, 'ARTIST')]
[(0, 45, 'SONG'), (48, 60, 'ARTIST')]
[(35, 53, 'SONG'), (0, 32, 'ARTIST')]
[(0, 7, 'SONG'), (10, 22, 'ARTIST')]
[(12, 47, 'SONG'), (0, 9, 'ARTIST')]
[(27, 64, 'SONG'), (0, 24, 'ARTIST')]
[(11, 25, 'SONG'), (0, 8, 'ARTIST')]
[(0, 23, 'SONG'), (26, 38, 'ARTIST')]
[(16, 47, 'SONG'), (0, 13, 'ARTIST')]
[(0, 8, 'SONG'), (11

[(9, 19, 'SONG'), (0, 6, 'ARTIST')]
[(12, 36, 'SONG'), (0, 9, 'ARTIST')]
[(0, 17, 'SONG'), (20, 31, 'ARTIST')]
[(13, 22, 'SONG'), (0, 10, 'ARTIST')]
[(11, 20, 'SONG'), (0, 8, 'ARTIST')]
[(12, 28, 'SONG'), (0, 9, 'ARTIST')]
[(0, 7, 'SONG'), (10, 16, 'ARTIST')]
[(0, 4, 'SONG'), (7, 18, 'ARTIST')]
[(0, 31, 'SONG'), (34, 48, 'ARTIST')]
[(36, 46, 'SONG'), (0, 33, 'ARTIST')]
[(0, 113, 'SONG'), (116, 119, 'ARTIST')]
[(42, 51, 'SONG'), (0, 39, 'ARTIST')]
[(8, 17, 'SONG'), (0, 5, 'ARTIST')]
[(0, 22, 'SONG'), (25, 40, 'ARTIST')]
[(45, 66, 'SONG'), (0, 42, 'ARTIST')]
[(0, 8, 'SONG'), (11, 19, 'ARTIST')]
[(5, 14, 'SONG'), (0, 2, 'ARTIST')]
[(14, 34, 'SONG'), (0, 11, 'ARTIST')]
[(0, 16, 'SONG'), (19, 34, 'ARTIST')]
[(16, 73, 'SONG'), (0, 13, 'ARTIST')]
[(8, 28, 'SONG'), (0, 5, 'ARTIST')]
[(13, 51, 'SONG'), (0, 10, 'ARTIST')]
[(0, 6, 'SONG'), (9, 24, 'ARTIST')]
[(15, 33, 'SONG'), (0, 12, 'ARTIST')]
[(0, 10, 'SONG'), (13, 26, 'ARTIST')]
[(16, 26, 'SONG'), (0, 13, 'ARTIST')]
[(0, 6, 'SONG'), (9, 19, '

In [135]:
trainData

[('Raphaël - Je Sais Que La Terre Est Plate (OFFICIAL MUSIC VIDEO)',
  {'entities': [(10, 40, 'SONG'), (0, 7, 'ARTIST')]}),
 ('On Efface - Julie Zenatti',
  {'entities': [(0, 9, 'SONG'), (12, 25, 'ARTIST')]}),
 ('The Baltimore Consort - Howells Delight',
  {'entities': [(24, 39, 'SONG'), (0, 21, 'ARTIST')]}),
 ('Martha Served - I Hate Sally',
  {'entities': [(0, 13, 'SONG'), (16, 28, 'ARTIST')]}),
 ('Zip-A-Dee-Doo-Dah (Song of the South) - Orlando Pops Orchestra',
  {'entities': [(0, 37, 'SONG'), (40, 62, 'ARTIST')]}),
 ('Liquid Time (composition by John Goodsall) - Brand X',
  {'entities': [(0, 42, 'SONG'), (45, 52, 'ARTIST')]}),
 ('Misery Path (From the Privilege of Evil) - Amorphis',
  {'entities': [(0, 40, 'SONG'), (43, 51, 'ARTIST')]}),
 ('Nuovi Re pt. I I (feat. Tek money - Lady Tambler) - Inoki',
  {'entities': [(0, 49, 'SONG'), (52, 57, 'ARTIST')]}),
 ('Halloween - Dead Kennedys',
  {'entities': [(0, 9, 'SONG'), (12, 25, 'ARTIST')]}),
 ('Brigada Victor Jara - Parto em terras di

In [53]:
# # setting up the pipeline and entity recognizer
# model = 'en_core_web_lg'
# if model is not None:
#     nlp = spacy.load(model)  # load existing spacy model
#     print("Loaded model '%s'" % model)
# else:
#     nlp = spacy.blank('en')  # create blank Language class
#     print("Created blank 'en' model")
# if 'ner' not in nlp.pipe_names:
#     include_pipes = ['ner'] #, 'parser', 'attribute_ruler','lemmatizer', 'Tok2Vec'
#     #nlp.add_pipe("tagger", source=spacy.load("en_core_web_lg"))
#     [nlp.create_pipe(pipe) for pipe in include_pipes] #ner = nlp.create_pipe('ner')
#     for pipe in include_pipes:
#         nlp.add_pipe(pipe)
#     print('1')
# else:
#     ner = nlp.get_pipe('ner')
#     print('2')

Loaded model 'en_core_web_lg'
2


In [54]:
# # Add new entity labels to entity recognizer
# for i, j in entities.items():
#     ner.add_label(i)
#     ner.add_label(j)
#     print(i, j)
# # Inititalizing optimizer
# if nlp is None:
#     optimizer = nlp.begin_training()
#     print('1')
# else:
#     optimizer = nlp.get_pipe("ner").create_optimizer()
#     print('2')

SONG ARTIST
2


In [40]:
# ?ner.labels

In [55]:
# # Get names of other pipes to disable them during training to train # only NER and update the weights
# from spacy.util import minibatch, compounding
# from spacy.training.example import Example


# n_iter = 5

# TRAIN_DATA = trainData

# other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
# with nlp.disable_pipes(*other_pipes):  # only train NER
#     for itn in range(n_iter):
#         random.shuffle(TRAIN_DATA)
#         losses = {}
#         batches = minibatch(TRAIN_DATA, 
#                             size=compounding(4., 32., 1.001)) 

#         for batch in batches:
#             texts, annotations = zip(*batch)
#             example = []
#             for i in range(len(texts)):
#                 doc = nlp.make_doc(text[i])
#                 example.append(Example.from_dict(doc, annotations[i])) # used this for reference https://stackoverflow.com/questions/66675261/how-can-i-work-with-example-for-nlp-update-problem-with-spacy3-0

#             # Updating the weights
#             nlp.update(example, sgd=optimizer, 
#                        drop=0.35, losses=losses)
#         print('Losses', losses)

/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "s" with entities "[(0, 42, 'SONG'), (42, 67, 'ARTIST')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "s" with entities "[(0, 44, 'SONG'), (44, 78, 'ARTIST')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "s

/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "s" with entities "[(0, 4, 'SONG'), (4, 25, 'ARTIST')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "l" with entities "[(0, 44, 'SONG'), (44, 65, 'ARTIST')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text " " 

/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "s" with entities "[(0, 42, 'SONG'), (42, 48, 'ARTIST')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "e" with entities "[(0, 26, 'SONG'), (26, 60, 'ARTIST')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "e

/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "s" with entities "[(0, 28, 'SONG'), (28, 53, 'ARTIST')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text " " with entities "[(0, 44, 'SONG'), (44, 52, 'ARTIST')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "e

/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "s" with entities "[(0, 25, 'SONG'), (25, 47, 'ARTIST')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "s" with entities "[(0, 9, 'SONG'), (9, 35, 'ARTIST')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "f" 

...

/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "n" with entities "[(0, 48, 'SONG'), (48, 58, 'ARTIST')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "n" with entities "[(0, 8, 'SONG'), (8, 16, 'ARTIST')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "V" 

/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "i" with entities "[(0, 54, 'SONG'), (54, 70, 'ARTIST')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "V" with entities "[(0, 77, 'SONG'), (77, 85, 'ARTIST')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "i

/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "c" with entities "[(0, 7, 'SONG'), (7, 26, 'ARTIST')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "c" with entities "[(0, 25, 'SONG'), (25, 41, 'ARTIST')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "s" 

Losses {'ner': 0.47577401995658875}


/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "s" with entities "[(0, 39, 'SONG'), (39, 68, 'ARTIST')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "c" with entities "[(0, 25, 'SONG'), (25, 32, 'ARTIST')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/petr/Documents/fun_stuff/youtube_app/utube/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "n

In [56]:
# test_docs = ["You're Beautiful - James Blunt",
#             "On Efface - Julie Zenatti",
#             "Hotel California - The Eagles"]
# test_docs_converted = [nlp(doc) for doc in raw_text[:100]]
# for converted_doc in test_docs_converted:
#     for ent in converted_doc.ents:
#         print(ent.label_, ent.text)
#     print('___________________')

SONG Je
SONG Sais
SONG Que
SONG La
SONG Est
SONG Plate
SONG Raphaël
___________________
SONG On
ORG Efface
SONG Julie
SONG Zenatti
___________________
SONG Delight
SONG The
SONG Consort
___________________
SONG Martha
SONG Served
SONG I
SONG Sally
___________________
SONG A
SONG Dee
SONG Doo
SONG Dah
SONG (
SONG Song
SONG the
LOC South
SONG )
SONG Orchestra
___________________
SONG Liquid
SONG Time
SONG (
SONG composition
SONG by
PERSON John Goodsall
SONG Brand
SONG X
___________________
SONG Path
SONG (
SONG the
SONG Privilege
SONG Evil
SONG )
___________________
SONG Re
SONG pt
SONG I
SONG (
SONG feat
SONG money
SONG Lady
SONG Tambler
SONG )
___________________
SONG Dead
___________________
SONG Parto
SONG em
SONG distantes
SONG Victor
___________________
SONG By
SONG Me
SONG Version
___________________
SONG Mistress
___________________
SONG What
SONG The
SONG Shadows
SONG Fall
___________________
SONG Life
SONG Force
SONG Daou
___________________
SONG The
SONG Dance
SONG Europe
SONG

## Following this tutorial: https://turbolab.in/build-a-custom-ner-model-using-spacy-3-0/

In [137]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import random

nlp = spacy.blank('en') # load a new spacy model
dbTrain = DocBin() # create a DocBin object
dbVal = DocBin()
dataTest = []

for text, annot in trainData: # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot['entities']: # add character indexes
        print(start, end, label)
        span = doc.char_span(start, end, label=label, alignment_mode='contract')
        if span is None:
            print('Skipping entity')
        else:
            ents.append(span)
    if random.random() <= 0.05:
        dataTest.append(text)
    elif random.random() <= 0.20:
        try:
            doc.ents = ents # label the text with the ents
            dbVal.add(doc)
        except:
            print(text, annot)
    else:
        try:
            doc.ents = ents # label the text with the ents
            dbTrain.add(doc)
        except:
            print(text, annot)
dbTrain.to_disk('/Users/petr/Documents/fun_stuff/youtube_app/data/ner/train.spacy') # save the docbin object
dbVal.to_disk('/Users/petr/Documents/fun_stuff/youtube_app/data/ner/val.spacy')

10 40 SONG
0 7 ARTIST
0 9 SONG
12 25 ARTIST
24 39 SONG
0 21 ARTIST
0 13 SONG
16 28 ARTIST
0 37 SONG
40 62 ARTIST
0 42 SONG
45 52 ARTIST
0 40 SONG
43 51 ARTIST
0 49 SONG
52 57 ARTIST
0 9 SONG
12 25 ARTIST
22 47 SONG
0 19 ARTIST
0 34 SONG
37 43 ARTIST
0 6 SONG
9 17 ARTIST
0 20 SONG
23 35 ARTIST
15 25 SONG
0 12 ARTIST
0 19 SONG
22 45 ARTIST
14 29 SONG
0 11 ARTIST
18 48 SONG
0 15 ARTIST
14 38 SONG
0 11 ARTIST
16 22 SONG
0 13 ARTIST
17 35 SONG
0 14 ARTIST
0 57 SONG
60 85 ARTIST
15 29 SONG
0 12 ARTIST
0 11 SONG
14 28 ARTIST
0 16 SONG
19 48 ARTIST
8 16 SONG
0 5 ARTIST
0 14 SONG
17 21 ARTIST
0 22 SONG
25 34 ARTIST
6 54 SONG
0 3 ARTIST
17 49 SONG
0 14 ARTIST
15 33 SONG
0 12 ARTIST
0 12 SONG
15 29 ARTIST
16 27 SONG
0 13 ARTIST
0 11 SONG
14 29 ARTIST
0 35 SONG
38 48 ARTIST
0 17 SONG
20 33 ARTIST
0 19 SONG
22 36 ARTIST
27 71 SONG
0 24 ARTIST
0 14 SONG
17 38 ARTIST
0 31 SONG
34 55 ARTIST
14 27 SONG
0 11 ARTIST
0 13 SONG
16 24 ARTIST
15 27 SONG
0 12 ARTIST
13 32 SONG
0 10 ARTIST
14 19 SONG
0 11 ARTI

14 25 SONG
0 11 ARTIST
0 33 SONG
36 41 ARTIST
0 7 SONG
10 21 ARTIST
0 27 SONG
30 39 ARTIST
28 49 SONG
0 25 ARTIST
0 17 SONG
20 38 ARTIST
14 38 SONG
0 11 ARTIST
13 33 SONG
0 10 ARTIST
0 17 SONG
20 37 ARTIST
0 19 SONG
22 31 ARTIST
14 28 SONG
0 11 ARTIST
0 6 SONG
9 21 ARTIST
12 27 SONG
0 9 ARTIST
15 19 SONG
0 12 ARTIST
0 15 SONG
18 36 ARTIST
16 26 SONG
0 13 ARTIST
0 28 SONG
31 48 ARTIST
0 6 SONG
9 19 ARTIST
0 22 SONG
25 46 ARTIST
15 24 SONG
0 12 ARTIST
0 11 SONG
14 52 ARTIST
0 9 SONG
12 24 ARTIST
14 21 SONG
0 11 ARTIST
12 24 SONG
0 9 ARTIST
0 8 SONG
11 23 ARTIST
23 46 SONG
0 20 ARTIST
0 26 SONG
29 34 ARTIST
0 22 SONG
25 36 ARTIST
0 14 SONG
17 25 ARTIST
15 26 SONG
0 12 ARTIST
15 34 SONG
0 12 ARTIST
0 41 SONG
44 54 ARTIST
17 34 SONG
0 14 ARTIST
0 8 SONG
11 22 ARTIST
0 6 SONG
9 19 ARTIST
0 13 SONG
16 22 ARTIST
21 35 SONG
0 18 ARTIST
12 44 SONG
0 9 ARTIST
0 12 SONG
15 27 ARTIST
0 25 SONG
28 46 ARTIST
14 31 SONG
0 11 ARTIST
11 27 SONG
0 8 ARTIST
20 53 SONG
0 17 ARTIST
0 33 SONG
36 48 ARTIST
17

Run this command in the terminal to train the model:
python -m spacy train config.cfg --output /Users/petr/Documents/fun_stuff/youtube_app/output --paths.train /Users/petr/Documents/fun_stuff/youtube_app/data/ner/train.spacy --paths.dev /Users/petr/Documents/fun_stuff/youtube_app/data/ner/val.spacy  

In [139]:
# loading and testing the model
import spacy
from spacy import displacy

nlp = spacy.load('output/model-last/') #load the model

print(f"Number of test examples: {len(dataTest)}")

test_docs_converted = [nlp(doc) for doc in dataTest]

for i in range(len(test_docs_converted)):
    converted_doc = test_docs_converted[i]
    text = dataTest[i]
    displacy.render(converted_doc, style='ent', jupyter=True)
    print(f"Actual text: {text}".format(text=text))
    print('___________________')



Number of test examples: 506


Actual text: Misery Path (From the Privilege of Evil) - Amorphis
___________________


Actual text: Fascinating Rhythm (Instrumental) (2007 Digital Remaster) - Billy May & His Orchestra
___________________


Actual text: Calm Da Seas - Dave Hollister
___________________


Actual text: Wohär dr Wind wäiht - Kisha
___________________


...

Actual text: Black Eyed Peas - Tell Your Momma Come (OFFICIAL MUSIC VIDEO)
___________________


Actual text: My Last Date (With You) - Skeeter Davis
___________________


Actual text: No Quarter - Crowbar
___________________


Actual text: Another Last Shot (Album Version) - Vivian
___________________


Actual text: Out Of This World - Black Flag
___________________


# IT WOOOOORKS! WOOOOOOO

Best model is about ~90% accurate

The plan now is to write a script that will take as input a directory with many songs in it, read the song names into a list, split the song title and artist using the above NER model, and relabel the mp3 metadata with the approrpriate song title and artist name using the mutahen library

In [5]:
musicDirectory = '/Users/petr/Documents/fun_stuff/youtube_app/data/Music'

In [6]:
for root,dirnames,fnames in os.walk(musicDirectory):
    print(len(fnames))

2177


In [127]:
for ent in doc.ents:
    print(doc)
    print(ent.label_, ent.text)

Ed Sheeran - Shape Of You [Official Lyric Video]
SONG Ed Sheeran


In [116]:
labels

{'SONG': 'Craig David', 'ARTIST': 'Insomnia'}

In [141]:
from mutagen.id3 import ID3NoHeaderError
from mutagen.id3 import ID3, TIT2, TALB, TPE1, TPE2, COMM, TCOM, TCON, TDRC, TRCK
import spacy

nlp = spacy.load('output/model-last/') #load the model

for root,dirnames,fnames in os.walk(musicDirectory):
    for fname in fnames:
        
        path = os.path.join(root, fname)
        
        text_data = fname[:-4]
        # Get the song and artist labels using our pretrained NER model
        doc = nlp(text_data)
        labels = {}
        for ent in doc.ents:
            labels[ent.label_] = ent.text
            
        # Read the ID3 tag or create one if not present
        try: 
            tags = ID3(path)
        except ID3NoHeaderError:
            print("Adding ID3 header")
            tags = ID3()
        try:
            tags["TIT2"] = TIT2(encoding=3, text=labels['SONG'])
        except:
            pass
        try:
            tags["TPE1"] = TPE1(encoding=3, text=labels['ARTIST'])
        except:
            pass
        
        path = os.path.join(root, fname)
        
        tags.save(path)